In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import subprocess
import sys
import io
import gradio as gr
import numpy as np
import random
import spaces
import torch
from diffusers import Flux2Pipeline, Flux2Transformer2DModel, FluxInpaintPipeline
from diffusers import BitsAndBytesConfig as DiffBitsAndBytesConfig
import requests
from PIL import Image
import json
import base64
from dotenv import load_dotenv
from huggingface_hub import login, InferenceClient
from helper_dev_utils import get_auto_logger
import sys
from pathlib import Path
import helper_plot_hangul
import importlib
import gc
import logging
import copy
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from matplotlib.patches import Rectangle
from typing import List, Optional, Tuple, Dict, Any
from helper_plot_hangul import matplotlib_font_reset, matplotlib_font_get
from helper_dev_utils import *
from diffusers.utils import load_image
logger = get_auto_logger()

In [3]:
# Hugging Face 로그인 (FLUX 모델 접근 권한 필요)
load_dotenv()

# 환경 변수에서 토큰 가져오기 또는 직접 입력
hf_token = os.getenv("HF_TOKEN")
logger.info(f"Hugging Face Token: {hf_token[:2]} ... {hf_token[-2:]}")

if hf_token:
    login(token=hf_token)
    print("✓ Logged in with HF_TOKEN from environment")
else:
    # 수동 로그인 (토큰 입력 필요)
    login()
    print("✓ Manual login completed")

2026-01-17 07:44:00 I [ipykernel_launcher:6] - Hugging Face Token: hf ... LA


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


✓ Logged in with HF_TOKEN from environment


In [9]:
import sys
from pathlib import Path

# 프로젝트 경로 설정
project_root = Path.cwd().parent.parent / "src" / "nanoCocoa_aiserver"
sys.path.insert(0, str(project_root))
logger.debug(project_root)

import torch
import gc
from PIL import Image
from typing import Optional

from diffusers import DiffusionPipeline
from config import TORCH_DTYPE
from services.monitor import flush_gpu


2026-01-17 07:46:32 D [ipykernel_launcher:7] - /home/spai0433/codeit-ai-3team-ad-content/src/nanoCocoa_aiserver


In [13]:
import torch
import warnings
from PIL import Image
from torchvision import transforms
from transformers import AutoModelForImageSegmentation
from config import DEVICE, MODEL_IDS
from utils import flush_gpu

# timm 라이브러리 deprecation 경고 억제
warnings.filterwarnings("ignore", category=FutureWarning, module="timm")

class SegmentationModel:
    """
    BiRefNet을 사용하여 이미지 세그멘테이션(배경 제거)을 수행하는 클래스입니다.
    """
    def __init__(self):
        self.device = DEVICE

    def run(self, image: Image.Image) -> tuple[Image.Image, Image.Image]:
        """
        이미지의 배경을 제거합니다.
        
        Args:
            image (Image.Image): 입력 이미지
            
        Returns:
            tuple[Image.Image, Image.Image]: (배경 제거된 이미지, 마스크)
        """
        logger.debug("[Engine] Loading BiRefNet... (BiRefNet 모델 로딩 중)")
        flush_gpu()
        
        model = AutoModelForImageSegmentation.from_pretrained(
            MODEL_IDS["SEG"], trust_remote_code=True
        ).to(self.device).eval()

        W, H = image.size
        # 고해상도 처리를 위해 리사이즈 (필요 시 조정 가능)
        img_resized = image.resize((1024, 1024), Image.LANCZOS)
        
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        
        input_tensor = transform(img_resized).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            preds = model(input_tensor)[-1].sigmoid().cpu()
        
        pred = preds[0].squeeze()
        mask = transforms.ToPILImage()(pred).resize((W, H), Image.LANCZOS)
        
        # 마스크 이진화 (Thresholding)
        mask = mask.point(lambda x: 255 if x > 128 else 0)
        
        # 마스크 블러 처리 (외곽선 부드럽게)
        from PIL import ImageFilter
        mask = mask.filter(ImageFilter.GaussianBlur(radius=5))
        
        result = image.copy()
        result.putalpha(mask)
        
        # 리소스 정리
        del model, input_tensor
        flush_gpu()
        
        return result, mask


In [6]:
"""
공간 분석 모듈: Qwen2-VL을 사용한 최적의 객체 배치 위치 탐지
Spatial Analysis Module: Detect optimal object placement using Qwen2-VL
"""

import torch
from PIL import Image, ImageDraw
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from qwen_vl_utils import process_vision_info
from typing import Union, Dict, List, Tuple
import re

from utils import flush_gpu


class SpatialAnalyzer:
    """
    Qwen2-VL을 사용하여 배경 이미지에서 최적의 객체 배치 위치를 찾는 클래스

    이 클래스는 Vision-Language 모델을 사용하여 표면을 감지하고,
    바운딩 박스를 결정하며, 객체 합성을 위한 마스크를 생성합니다.

    Attributes:
        model_name (str): 사용할 HuggingFace 모델 이름
        device (str): 모델을 실행할 디바이스
        model: Qwen2-VL 모델 인스턴스
        processor: 이미지/텍스트 전처리 프로세서
    """

    def __init__(
        self, model_name: str = "Qwen/Qwen2-VL-7B-Instruct", device: str = None
    ):
        """
        SpatialAnalyzer 초기화

        Args:
            model_name: HuggingFace 모델 식별자 (기본값: Qwen2-VL-7B-Instruct)
            device: 모델 실행 디바이스 ('cuda' 또는 'cpu', 기본값: 자동 감지)
        """
        self.model_name = model_name
        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.model = None
        self.processor = None

        print(f"🔧 SpatialAnalyzer 초기화: {model_name}")

    def _load_model(self):
        """Qwen2-VL 모델과 프로세서를 로드합니다."""
        if self.model is None:
            print(f"  Qwen2-VL 모델을 {self.device}에 로드 중...")

            # Vision-Language 모델 로드
            self.model = Qwen2VLForConditionalGeneration.from_pretrained(
                self.model_name,
                torch_dtype=torch.bfloat16,  # 메모리 절약
                device_map="auto",  # 자동 디바이스 배치
            )

            # 이미지와 텍스트 전처리를 위한 프로세서 로드
            self.processor = AutoProcessor.from_pretrained(self.model_name)

            print(f"  ✓ Qwen2-VL 모델 로드 완료")

    def _unload_model(self):
        """VRAM 확보를 위해 모델을 언로드합니다."""
        if self.model is not None:
            print("  Qwen2-VL 모델 언로드 중...")
            # device_map="auto"로 로드된 모델은 .to("cpu") 사용 불가
            del self.model
            del self.processor
            self.model = None
            self.processor = None
            flush_gpu()  # GPU 캐시 정리

    def detect_surface(
        self,
        image: Union[Image.Image, str],
        query: str = "Find the flat surface where I can place an object. Return the bounding box coordinates.",
        auto_unload: bool = True,
    ) -> Dict[str, any]:
        """
        객체 배치를 위한 최적의 표면을 탐지합니다.

        Vision-Language 모델에 이미지와 질문을 제공하여
        객체를 놓을 수 있는 최적의 위치를 찾습니다.

        Args:
            image: PIL Image 객체 또는 이미지 경로
            query: VL 모델에 물어볼 질문 (위치 탐지 요청)

        Returns:
            다음을 포함하는 딕셔너리:
                - 'bbox': [x1, y1, x2, y2] 정규화된 좌표 (0-1000 범위)
                - 'text': 모델의 전체 응답 텍스트
                - 'image_size': 입력 이미지의 (width, height)

        Example:
            >>> analyzer = SpatialAnalyzer()
            >>> result = analyzer.detect_surface(
            ...     bg_image,
            ...     "맥주병을 놓을 테이블 중앙을 찾아주세요"
            ... )
            >>> bbox = result['bbox']  # [x1, y1, x2, y2]
        """
        try:
            # 모델 로드 (필요시)
            self._load_model()

            # 경로가 제공된 경우 이미지 로드
            if isinstance(image, str):
                from .utils import load_image

                image = load_image(image)

            image_size = image.size  # (width, height)

            print(f"  이미지 분석 중 ({image_size[0]}x{image_size[1]})...")
            print(f"  질문: {query}")

            # 모델을 위한 메시지 준비 (멀티모달 입력)
            messages = [
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "image",
                            "image": image,  # 이미지 입력
                        },
                        {"type": "text", "text": query},  # 텍스트 질문
                    ],
                }
            ]

            # 입력 전처리
            text = self.processor.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            image_inputs, video_inputs = process_vision_info(messages)

            inputs = self.processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )
            inputs = inputs.to(self.device)

            # 응답 생성 (추론)
            with torch.no_grad():  # 그래디언트 계산 비활성화
                generated_ids = self.model.generate(
                    **inputs, max_new_tokens=256  # 최대 응답 길이
                )

            # 입력 토큰 제거 (생성된 부분만 추출)
            generated_ids_trimmed = [
                out_ids[len(in_ids) :]
                for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]

            # 응답 디코딩 (토큰 -> 텍스트)
            output_text = self.processor.batch_decode(
                generated_ids_trimmed,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False,
            )[0]

            print(f"  모델 응답: {output_text[:100]}...")

            # 응답에서 바운딩 박스 파싱
            bbox = self._parse_bbox(output_text)

            result = {"bbox": bbox, "text": output_text, "image_size": image_size}

            print(f"  ✓ 표면 탐지 완료: {bbox}")

            return result

        finally:
            # VRAM 확보를 위해 항상 모델 언로드
            if auto_unload:
                self._unload_model()

    def _parse_bbox(self, text: str) -> List[int]:
        """
        모델 출력에서 바운딩 박스 좌표를 파싱합니다.

        Qwen-VL은 일반적으로 다음 형식으로 bbox를 반환합니다:
        - 형식 1: <|box_start|>(x1,y1),(x2,y2)<|box_end|>
        - 형식 2: 텍스트 내의 숫자 나열

        Returns:
            정규화된 좌표의 [x1, y1, x2, y2] (0-1000 범위)
        """
        # 패턴 1: <|box_start|>(x1,y1),(x2,y2)<|box_end|> 형태 찾기
        box_pattern = r"<\|box_start\|\>\((\d+),(\d+)\),\((\d+),(\d+)\)<\|box_end\|\>"
        match = re.search(box_pattern, text)

        if match:
            return [
                int(match.group(1)),
                int(match.group(2)),
                int(match.group(3)),
                int(match.group(4)),
            ]

        # 패턴 2: 콤마로 구분된 4개의 숫자 찾기
        numbers = re.findall(r"\b\d+\b", text)
        if len(numbers) >= 4:
            # 처음 4개 숫자를 bbox로 사용
            return [int(numbers[0]), int(numbers[1]), int(numbers[2]), int(numbers[3])]

        # 기본값: 파싱 실패 시 중앙 영역 사용
        print("  ⚠ bbox 파싱 실패, 중앙 영역 사용")
        return [400, 400, 600, 600]  # 중앙 영역 (정규화 0-1000)

    def create_mask(
        self,
        image_size: Tuple[int, int],
        bbox: List[int],
        mask_color: int = 255,
        background_color: int = 0,
    ) -> Image.Image:
        """
        바운딩 박스 좌표로부터 이진 마스크를 생성합니다.

        Args:
            image_size: 대상 이미지의 (width, height)
            bbox: 정규화된 좌표의 [x1, y1, x2, y2] (0-1000 범위)
            mask_color: 마스크 영역의 색상 (기본값: 255 = 흰색)
            background_color: 배경 색상 (기본값: 0 = 검정)

        Returns:
            이진 마스크 이미지 ('L' 모드의 PIL.Image)

        Example:
            >>> mask = analyzer.create_mask((1024, 1024), [400, 400, 600, 600])
        """
        width, height = image_size

        # 정규화된 좌표(0-1000)를 픽셀 좌표로 변환
        x1 = int(bbox[0] * width / 1000)
        y1 = int(bbox[1] * height / 1000)
        x2 = int(bbox[2] * width / 1000)
        y2 = int(bbox[3] * height / 1000)

        # 검은 배경 생성
        mask = Image.new("L", image_size, background_color)
        draw = ImageDraw.Draw(mask)

        # 마스크 영역에 흰색 사각형 그리기
        draw.rectangle([x1, y1, x2, y2], fill=mask_color)

        print(
            f"  ✓ 마스크 생성 완료: {image_size[0]}x{image_size[1]}, "
            f"영역: ({x1},{y1})-({x2},{y2})"
        )

        return mask

    def visualize_bbox(
        self, image: Image.Image, bbox: List[int], color: str = "red", width: int = 3
    ) -> Image.Image:
        """
        이미지에 바운딩 박스를 시각화합니다.

        Args:
            image: 입력 PIL Image
            bbox: 정규화된 좌표의 [x1, y1, x2, y2] (0-1000 범위)
            color: 바운딩 박스 색상
            width: 선 두께

        Returns:
            바운딩 박스가 그려진 이미지

        Example:
            >>> bbox_img = analyzer.visualize_bbox(bg_image, [400, 400, 600, 600])
        """
        img_copy = image.copy()
        draw = ImageDraw.Draw(img_copy)

        # 정규화된 좌표를 픽셀 좌표로 변환
        img_width, img_height = image.size
        x1 = int(bbox[0] * img_width / 1000)
        y1 = int(bbox[1] * img_height / 1000)
        x2 = int(bbox[2] * img_width / 1000)
        y2 = int(bbox[3] * img_height / 1000)

        # 사각형 그리기
        draw.rectangle([x1, y1, x2, y2], outline=color, width=width)

        return img_copy

    def generate_flux_prompt(
        self,
        image: Union[Image.Image, str],
        query: str = None,
        detail_level: str = "detailed",
        language: str = "en",
        auto_unload: bool = True,
    ) -> Dict[str, str]:
        """
        이미지를 분석하여 FLUX.1용 상세 프롬프트를 생성합니다.

        Args:
            image: PIL Image 객체 또는 이미지 경로
            query: 커스텀 질문 (None이면 기본 프롬프트 사용)
            detail_level: 'simple', 'detailed', 'ultra' 중 선택
            language: 'en' (영어) 또는 'ko' (한국어)

        Returns:
            딕셔너리:
                - 'prompt': FLUX.1용 생성 프롬프트
                - 'raw_response': 모델의 원본 응답
                - 'image_size': 입력 이미지의 (width, height)

        Example:
            >>> result = analyzer.generate_flux_prompt(image, detail_level="ultra")
            >>> flux_prompt = result['prompt']
        """
        try:
            self._load_model()

            # 경로가 제공된 경우 이미지 로드
            if isinstance(image, str):
                image = Image.open(image).convert("RGB")

            image_size = image.size

            # detail_level에 따른 기본 질문 설정
            if query is None:
                detail_prompts = {
                    "simple": (
                        "Describe this image briefly for text-to-image generation."
                        if language == "en"
                        else "이미지 생성을 위해 이 이미지를 간단히 설명해주세요."
                    ),
                    "detailed": (
                        "Analyze this image in detail for FLUX.1 text-to-image generation. "
                        "Include: main subject, composition, colors, lighting, mood, style, "
                        "and any important visual elements. Format as a single descriptive prompt."
                        if language == "en"
                        else "FLUX.1 이미지 생성을 위해 이 이미지를 상세히 분석해주세요. "
                        "포함 사항: 주요 피사체, 구도, 색감, 조명, 분위기, 스타일, 중요한 시각적 요소. "
                        "하나의 설명 프롬프트 형식으로 작성해주세요."
                    ),
                    "ultra": (
                            # [English] - 어종 판별 로직(Forensic Taxonomy)을 추가하여 정확한 형태(Shape)를 유도합니다.
                            "Act as a 'Forensic Seafood Taxonomist & Visual Tagger'. "
                            "Your goal is to identify the EXACT SPECIES based on visual anatomy (tail shape, skin pattern) "
                            "to ensure FLUX.1 draws the correct skeletal structure. "
                            
                            "Analyze the image and output a comma-separated tag list following this priority logic: "
                            
                            "1. [SPECIES IDENTIFICATION] (CRITICAL): "
                            "   - Look at the Tail & Body: "
                            "     * Forked/V-shape tail + Golden skin = Tag: 'dried young pollack (nogari), dried alaska pollock'. "
                            "     * Round/Oval pressed shape = Tag: 'dried filefish fillet (jwipo)'. "
                            "     * Tentacles/Triangular head = Tag: 'dried squid, cuttlefish'. "
                            "     * Silver skin + Spiky fins = Tag: 'dried gizzard shad'. "
                            "   - If unsure, use safe fallback: 'roasted dried fish jerky'. "
                            
                            "2. [PHYSICAL STATE] (Must modify species): "
                            "   - Tag: 'headless torso', 'flattened body', 'shriveled skin', 'stiff texture'. "
                            "   - Constraint: 'no fresh scales', 'no slime', 'no eyes'. "
                            
                            "3. [TEXTURE & COLOR]: "
                            "   - Tag: 'matte parchment-like finish', 'fibrous meat', 'golden-brown toasted', 'char marks'. "
                            
                            "4. [PLATING & CONTEXT]: "
                            "   - Tag: 'rectangular plate', 'mayonnaise dip with chili', 'roasted peanuts', 'pub lighting'. "
                            
                            "5. [STYLE]: "
                            "   - Tag: 'macro food photography', 'realistic texture', '8k', 'top-down view'. "
                            
                            "Final Output: A single list of tags starting with the identified species."
                            if language == "en"
                            
                            # [Korean] - 한글 지침
                            else "FLUX.1을 위한 '해산물 법의학 분석가 및 태거'로 행동하세요. "
                            "시각적 해부학(꼬리 모양, 껍질 패턴)을 분석하여 '정확한 어종(Species)'을 식별하는 것이 목표입니다. "
                            "그래야 AI가 뼈대와 형태를 정확히 그릴 수 있습니다. "
                            
                            "다음 우선순위 로직에 따라 분석하고 쉼표로 구분된 태그 리스트를 출력하세요: "
                            
                            "1. [어종 정밀 판별] (핵심): "
                            "   - 꼬리와 몸통을 관찰하세요: "
                            "     * 갈라진 V자 꼬리 + 황금빛 껍질 = 태그: 'dried young pollack (nogari), dried alaska pollock' (노가리/명태). "
                            "     * 둥글고 납작한 압축 형태 = 태그: 'dried filefish fillet (jwipo)' (쥐포). "
                            "     * 다리/삼각형 머리 = 태그: 'dried squid' (오징어). "
                            "     * 은색 껍질 + 뾰족한 지느러미 = 태그: 'dried gizzard shad' (전어). "
                            "   - 확신이 없으면 안전한 태그 사용: 'roasted dried fish jerky' (구운 건어물). "
                            
                            "2. [물리적 상태] (어종 수식): "
                            "   - 태그: '머리 없는 몸통(headless)', '납작해진 몸체', '쪼그라든', '뻣뻣한 질감'. "
                            "   - 제약: '싱싱한 비늘 없음', '점액 없음', '눈알 없음'. "
                            
                            "3. [질감 및 색상]: "
                            "   - 태그: '매트한 양피지 같은 마감', '섬유질 살결', '황금빛 갈색 구이', '그을린 자국'. "
                            
                            "4. [플레이팅 및 맥락]: "
                            "   - 태그: '직사각형 접시', '고추 마요네즈 소스', '구운 땅콩', '술집 조명'. "
                            
                            "5. [스타일]: "
                            "   - 태그: '매크로 음식 사진', '사실적 질감', '8k', '탑다운 뷰'. "
                            
                            "최종 출력: 식별된 어종 태그로 시작하는 단일 태그 리스트."
                        )
                }
                query = detail_prompts.get(detail_level, detail_prompts["detailed"])

            print(f"  이미지 프롬프트 생성 중 (레벨: {detail_level})...")

            # 모델을 위한 메시지 준비
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": image},
                        {"type": "text", "text": query},
                    ],
                }
            ]

            # 입력 전처리
            text = self.processor.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            image_inputs, video_inputs = process_vision_info(messages)

            inputs = self.processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )
            inputs = inputs.to(self.device)

            # 응답 생성 (detail_level에 따라 토큰 수 조정)
            max_tokens = {"simple": 128, "detailed": 256, "ultra": 512}
            
            with torch.no_grad():
                generated_ids = self.model.generate(
                    **inputs,
                    max_new_tokens=max_tokens.get(detail_level, 256),
                    temperature=0.7,
                    top_p=0.9
                )

            # 응답 디코딩
            generated_ids_trimmed = [
                out_ids[len(in_ids):]
                for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]

            output_text = self.processor.batch_decode(
                generated_ids_trimmed,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False,
            )[0]

            # 프롬프트 정제 (불필요한 문장 제거)
            prompt = self._refine_prompt(output_text)

            result = {
                "prompt": prompt,
                "raw_response": output_text,
                "image_size": image_size
            }

            print(f"  ✓ 프롬프트 생성 완료 (길이: {len(prompt)} chars)")
            print(f"  프롬프트 미리보기: {prompt[:100]}...")

            return result

        finally:
            if auto_unload:
                self._unload_model()

    def generate_flux_prompt_from_keywords(
        self,
        keywords: Union[str, List[str]],
        style: str = "photorealistic",
        detail_level: str = "ultra",
        language: str = "en",
        auto_unload: bool = True,
    ) -> Dict[str, str]:
        """
        키워드만으로 FLUX.1용 상세 프롬프트를 생성합니다.
        
        Args:
            keywords: 단일 문자열 또는 키워드 리스트 (예: "beer, pub" 또는 ["beer", "pub", "night"])
            style: 'photorealistic', 'artistic', 'cinematic', 'minimalist' 중 선택
            detail_level: 'simple', 'detailed', 'ultra' 중 선택
            language: 'en' (영어) 또는 'ko' (한국어)
            
        Returns:
            딕셔너리:
                - 'prompt': FLUX.1용 생성 프롬프트
                - 'raw_response': 모델의 원본 응답
                - 'keywords': 입력된 키워드
                
        Example:
            >>> result = analyzer.generate_flux_prompt_from_keywords(
            ...     "beer, wooden table, warm lighting",
            ...     style="cinematic",
            ...     detail_level="ultra"
            ... )
            >>> flux_prompt = result['prompt']
        """
        try:
            self._load_model()
            
            # 키워드 정규화
            if isinstance(keywords, str):
                keywords_list = [k.strip() for k in keywords.split(',')]
            else:
                keywords_list = keywords
            
            keywords_str = ", ".join(keywords_list)
            
            # 스타일별 기본 설정
            style_guides = {
                "photorealistic": {
                    "en": "professional photography, sharp focus, 8k resolution, highly detailed",
                    "ko": "전문 사진, 선명한 초점, 8k 해상도, 매우 상세함"
                },
                "artistic": {
                    "en": "artistic composition, painterly quality, creative lighting, masterpiece",
                    "ko": "예술적 구도, 회화적 품질, 창의적 조명, 걸작"
                },
                "cinematic": {
                    "en": "cinematic lighting, dramatic atmosphere, film grain, anamorphic lens",
                    "ko": "영화적 조명, 극적인 분위기, 필름 그레인, 아나모픽 렌즈"
                },
                "minimalist": {
                    "en": "minimalist composition, clean lines, soft shadows, elegant simplicity",
                    "ko": "미니멀 구도, 깔끔한 선, 부드러운 그림자, 우아한 단순함"
                }
            }
            
            style_suffix = style_guides.get(style, style_guides["photorealistic"])[language]
            
            # detail_level에 따른 질문 구성
            if language == "en":
                if detail_level == "simple":
                    query = (
                        f"Create a brief FLUX.1 text-to-image prompt based on these keywords: {keywords_str}. "
                        f"Style: {style}. Write a concise, single-paragraph prompt suitable for image generation."
                    )
                elif detail_level == "detailed":
                    query = (
                        f"You are a professional FLUX.1 prompt engineer. Create a highly detailed text-to-image prompt "
                        f"using these keywords: {keywords_str}. Target style: {style}.\n\n"
                        f"Your prompt must be rich, descriptive, and include:\n"
                        f"- Main subject: Describe appearance, materials, textures, colors in detail\n"
                        f"- Spatial layout: Foreground, midground, background elements and their relationships\n"
                        f"- Lighting: Type (natural/artificial), direction, quality (soft/hard), color temperature\n"
                        f"- Color palette: Dominant colors, accent colors, color harmony\n"
                        f"- Atmosphere: Mood, feeling, time of day, weather conditions\n"
                        f"- Technical aspects: {style_suffix}\n\n"
                        f"Write as a flowing, narrative paragraph. DO NOT use bullet points or lists. "
                        f"Output ONLY the final prompt text."
                    )
                else:  # ultra
                    query = (
                        f"You are a master FLUX.1 prompt engineer creating a production-grade, "
                        f"ultra-detailed text-to-image prompt from these keywords: {keywords_str}. "
                        f"Target style: {style}.\n\n"
                        f"Create an EXTREMELY DETAILED prompt with vivid, specific descriptions:\n\n"
                        f"1. SUBJECT DETAILS:\n"
                        f"   - Exact appearance: shape, size, proportions\n"
                        f"   - Surface qualities: texture (rough/smooth/glossy), material properties, patterns\n"
                        f"   - Colors: specific hues, saturation levels, color variations, highlights/shadows\n"
                        f"   - Fine details: scratches, reflections, transparency, wear marks\n\n"
                        f"2. SPATIAL COMPOSITION:\n"
                        f"   - Camera angle and perspective (bird's eye, eye-level, low angle, Dutch tilt)\n"
                        f"   - Depth layers: foreground (closest), midground, background (farthest)\n"
                        f"   - Rule of thirds positioning, leading lines, symmetry/asymmetry\n"
                        f"   - Depth of field: what's in focus vs. bokeh blur\n\n"
                        f"3. LIGHTING DESIGN:\n"
                        f"   - Light source type: sunlight, studio lights, ambient, practical lights\n"
                        f"   - Direction: front-lit, back-lit, side-lit, rim lighting\n"
                        f"   - Quality: hard shadows vs. soft diffused light\n"
                        f"   - Color temperature: warm (golden hour) vs. cool (blue hour)\n"
                        f"   - Special effects: god rays, lens flare, volumetric fog, caustics\n\n"
                        f"4. COLOR & ATMOSPHERE:\n"
                        f"   - Primary color scheme and palette\n"
                        f"   - Accent colors for visual interest\n"
                        f"   - Color grading: vibrant, desaturated, high contrast, muted\n"
                        f"   - Atmospheric conditions: clear, hazy, misty, dusty, smoky\n"
                        f"   - Emotional mood: peaceful, dramatic, mysterious, energetic, melancholic\n\n"
                        f"5. TECHNICAL PHOTOGRAPHY:\n"
                        f"   - {style_suffix}\n"
                        f"   - Lens characteristics: focal length, aperture effect, lens distortion\n"
                        f"   - Image quality: resolution, sharpness, grain/noise level\n"
                        f"   - Post-processing style: color grading, contrast adjustments\n\n"
                        f"Write as ONE comprehensive, flowing narrative paragraph that paints a complete visual picture. "
                        f"Be specific, vivid, and detailed. Use descriptive adjectives and sensory language. "
                        f"DO NOT write as a list or analysis. Output ONLY the raw prompt string ready for FLUX.1."
                    )
            else:  # Korean
                if detail_level == "simple":
                    query = (
                        f"다음 키워드로 간단한 FLUX.1 이미지 생성 프롬프트를 만들어주세요: {keywords_str}. "
                        f"스타일: {style}. 간결한 한 문단으로 작성하세요."
                    )
                elif detail_level == "detailed":
                    query = (
                        f"당신은 전문 FLUX.1 프롬프트 엔지니어입니다. 다음 키워드로 매우 상세한 이미지 생성 프롬프트를 "
                        f"작성하세요: {keywords_str}. 목표 스타일: {style}.\n\n"
                        f"프롬프트는 풍부하고 구체적이어야 하며 다음을 포함해야 합니다:\n"
                        f"- 주요 피사체: 외형, 재질, 질감, 색상을 상세히 묘사\n"
                        f"- 공간 배치: 전경, 중경, 배경 요소들과 그 관계\n"
                        f"- 조명: 종류(자연광/인공광), 방향, 품질(강함/부드러움), 색온도\n"
                        f"- 색상 팔레트: 주조색, 강조색, 색상 조화\n"
                        f"- 분위기: 무드, 느낌, 시간대, 날씨 조건\n"
                        f"- 기술적 요소: {style_suffix}\n\n"
                        f"흐르는 서술형 문단으로 작성하세요. 글머리 기호나 목록 형식을 사용하지 마세요. "
                        f"완성된 프롬프트 텍스트만 출력하세요."
                    )
                else:  # ultra
                    query = (
                        f"당신은 최고 수준의 FLUX.1 프롬프트 엔지니어입니다. 다음 키워드로 프로덕션 급의 "
                        f"초상세 이미지 생성 프롬프트를 작성하세요: {keywords_str}. "
                        f"목표 스타일: {style}.\n\n"
                        f"생동감 있고 구체적인 묘사로 극도로 상세한 프롬프트를 작성하세요:\n\n"
                        f"1. 피사체 상세 묘사:\n"
                        f"   - 정확한 외형: 형태, 크기, 비율\n"
                        f"   - 표면 특성: 질감(거칠기/매끄러움/광택), 재질 속성, 패턴\n"
                        f"   - 색상: 구체적인 색조, 채도 수준, 색상 변화, 하이라이트/그림자\n"
                        f"   - 미세 디테일: 긁힌 자국, 반사, 투명도, 마모 흔적\n\n"
                        f"2. 공간 구성:\n"
                        f"   - 카메라 각도와 원근법(조감도, 눈높이, 로우앵글, 더치앵글)\n"
                        f"   - 심도 레이어: 전경(가장 가까운), 중경, 배경(가장 먼)\n"
                        f"   - 삼분할법 배치, 유도선, 대칭/비대칭\n"
                        f"   - 피사계 심도: 초점이 맞는 부분 vs. 보케 블러\n\n"
                        f"3. 조명 설계:\n"
                        f"   - 광원 종류: 햇빛, 스튜디오 조명, 주변광, 실용 조명\n"
                        f"   - 방향: 정면광, 역광, 측면광, 림 라이팅\n"
                        f"   - 품질: 강한 그림자 vs. 부드럽게 확산된 빛\n"
                        f"   - 색온도: 따뜻한 빛(골든아워) vs. 차가운 빛(블루아워)\n"
                        f"   - 특수 효과: 빛줄기, 렌즈 플레어, 볼류메트릭 안개, 코스틱스\n\n"
                        f"4. 색상 및 분위기:\n"
                        f"   - 주요 색상 구성과 팔레트\n"
                        f"   - 시각적 흥미를 위한 강조색\n"
                        f"   - 색 보정: 선명한, 탈색된, 고대비, 음소거된\n"
                        f"   - 대기 상태: 맑은, 흐릿한, 안개 낀, 먼지 낀, 연기 자욱한\n"
                        f"   - 감정적 무드: 평화로운, 극적인, 신비로운, 활기찬, 우울한\n\n"
                        f"5. 기술적 촬영 요소:\n"
                        f"   - {style_suffix}\n"
                        f"   - 렌즈 특성: 초점거리, 조리개 효과, 렌즈 왜곡\n"
                        f"   - 이미지 품질: 해상도, 선명도, 그레인/노이즈 수준\n"
                        f"   - 후처리 스타일: 색 보정, 대비 조정\n\n"
                        f"완전한 시각적 그림을 그리는 하나의 포괄적이고 흐르는 서술형 문단으로 작성하세요. "
                        f"구체적이고, 생동감 있고, 상세하게 작성하세요. 묘사적인 형용사와 감각적 언어를 사용하세요. "
                        f"목록이나 분석문 형식으로 작성하지 마세요. FLUX.1에 바로 사용 가능한 완성된 프롬프트 문자열만 출력하세요."
                    )
            
            print(f"  키워드 기반 프롬프트 생성 중...")
            print(f"  키워드: {keywords_str}")
            print(f"  스타일: {style}, 레벨: {detail_level}")
            
            # 텍스트만으로 메시지 준비 (이미지 없음)
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": query}
                    ]
                }
            ]
            
            # 입력 전처리
            text = self.processor.apply_chat_template(
                messages, tokenize=False, add_generation_prompt=True
            )
            
            inputs = self.processor(
                text=[text],
                padding=True,
                return_tensors="pt"
            )
            inputs = inputs.to(self.device)
            
            # 응답 생성
            max_tokens = {"simple": 128, "detailed": 256, "ultra": 512}
            
            with torch.no_grad():
                generated_ids = self.model.generate(
                    **inputs,
                    max_new_tokens=max_tokens.get(detail_level, 256),
                    temperature=0.8,  # 창의성을 위해 약간 높임
                    top_p=0.9
                )
            
            # 응답 디코딩
            generated_ids_trimmed = [
                out_ids[len(in_ids):]
                for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            
            output_text = self.processor.batch_decode(
                generated_ids_trimmed,
                skip_special_tokens=True,
                clean_up_tokenization_spaces=False
            )[0]
            
            # 프롬프트 정제
            prompt = self._refine_prompt(output_text)
            
            result = {
                "prompt": prompt,
                "raw_response": output_text,
                "keywords": keywords_str,
                "style": style
            }
            
            print(f"  ✓ 프롬프트 생성 완료 (길이: {len(prompt)} chars)")
            print(f"  프롬프트 미리보기: {prompt[:100]}...")
            
            return result
            
        finally:
            if auto_unload:
                self._unload_model()

    def _refine_prompt(self, text: str) -> str:
        """
        모델 응답을 FLUX.1용 프롬프트로 정제합니다.
        
        Args:
            text: 모델의 원본 응답
            
        Returns:
            정제된 프롬프트 문자열
        """
        # 불필요한 접두사 제거
        prefixes_to_remove = [
            "This image shows ",
            "The image depicts ",
            "In this image, ",
            "이 이미지는 ",
            "이미지에는 "
        ]
        
        refined = text.strip()
        for prefix in prefixes_to_remove:
            if refined.startswith(prefix):
                refined = refined[len(prefix):]
        
        # 첫 글자 대문자로
        if refined:
            refined = refined[0].upper() + refined[1:]
        
        return refined

    def __del__(self):
        """객체 소멸 시 정리 작업"""
        self._unload_model()


In [14]:
from models.flux_generator import FluxGenerator

fluxgenerator = FluxGenerator()
segmenter = SegmentationModel()
analyzer = SpatialAnalyzer()
logger.debug("로딩 성공")

2026-01-17 07:47:28 I [flux_generator:37] - FluxGenerator initialized (pipelines will load on demand)
🔧 SpatialAnalyzer 초기화: Qwen/Qwen2-VL-7B-Instruct
2026-01-17 07:47:28 D [ipykernel_launcher:6] - 로딩 성공
🔧 SpatialAnalyzer 초기화: Qwen/Qwen2-VL-7B-Instruct
2026-01-17 07:47:28 D [ipykernel_launcher:6] - 로딩 성공


In [15]:
product_fg, product_mask = segmenter.run(Image.open("image01.png"))

2026-01-17 07:47:30 D [ipykernel_launcher:29] - [Engine] Loading BiRefNet... (BiRefNet 모델 로딩 중)


In [ ]:
resutl = analyzer.generate_flux_prompt("image01.png", detail_level="ultra")
logger.info(resutl['prompt'])

In [ ]:
prompt="""A detailed, photorealistic close-up shot on a worn wooden pub table. Two glasses of Korean draft beer are central. On the left, a tall, slender pilsner glass for a woman, filled with golden beer and a thick foam head. On the right, a robust, handled glass mug for a man, also filled. Both glasses are heavily covered in cold condensation and realistic water droplets streaking down the sides, pooling slightly on the table. The labels on the glasses are partially obscured by moisture but suggest popular Korean brands like 'Cass' or 'Terra'. In the background, out of focus, a lively, warm-lit Korean pub (pocha) interior with blurred patrons, metal chopsticks, and small snack dishes. Natural bar lighting, highly textured, cinematic quality, 8k resolution."""
image = fluxgenerator.generate_background(f"{resutl['prompt']}")
analyzer._unload_model()
image.show()

In [ ]:
from PIL import Image
from models.flux_generator import FluxGenerator

# 1. FluxGenerator 초기화
generator = FluxGenerator()

# 2. 흐린 한국 맥주집 배경 생성 (만화풍)
background_prompt = """
A blurred, dreamy illustration of a cozy Korean beer pub (pocha) interior. 
Cartoon style, anime aesthetic, soft focus background with warm ambient lighting. 
Blurred hanging lanterns, wooden tables, and traditional decorations. 
Empty foreground space for product placement. 
Bokeh effect, shallow depth of field, cinematic atmosphere, 8k illustration.
"""

bg_image = generator.generate_background(
    prompt=background_prompt,
    guidance_scale=3.5,
    seed=42
)

# 3. 노가리 이미지 로드 (배경 제거된 RGBA)
nogari_image = Image.open("image01.png").convert("RGBA")
nogari_mask = nogari_image.split()[3]  # 알파 채널을 마스크로 사용

# 4. 노가리를 선명하게 유지하면서 배경에 합성
product_prompt = """
Crystal clear, ultra-sharp dried yellow corvina fish (nogari) on a white plate. 
Photorealistic detail with visible texture, char marks, and natural sheen. 
Sharp focus on product, contrasting with blurred cartoon background. 
Professional food photography lighting, 8k, masterpiece quality.
"""

final_image = generator.inject_features_via_inpaint(
    background=bg_image,
    product_foreground=nogari_image,
    product_mask=nogari_mask,
    position=(200, 300),  # 배치 위치 조정
    prompt=product_prompt,
    negative_prompt="blurry product, soft focus product, cartoon fish, unrealistic",
    strength=0.3,  # 낮은 값 = 노가리 원본 유지
    guidance_scale=4.0,
    seed=42
)

#final_image.save("nogari_ad.png")
final_image.show()

2026-01-17 07:47:43 I [flux_generator:37] - FluxGenerator initialized (pipelines will load on demand)
2026-01-17 07:47:43 I [flux_generator:139] - [FluxGenerator] Generating background with Text-to-Image...
2026-01-17 07:47:43 I [flux_generator:60] - [FluxGenerator] Loading FLUX Text-to-Image pipeline...
2026-01-17 07:47:43 I [flux_generator:44] - [FluxGenerator] Loading 8bit quantized Transformer...
2026-01-17 07:47:43 I [flux_generator:139] - [FluxGenerator] Generating background with Text-to-Image...
2026-01-17 07:47:43 I [flux_generator:60] - [FluxGenerator] Loading FLUX Text-to-Image pipeline...
2026-01-17 07:47:43 I [flux_generator:44] - [FluxGenerator] Loading 8bit quantized Transformer...


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

2026-01-17 07:50:02 I [flux_generator:52] - [FluxGenerator] Transformer loaded with 8bit quantization


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

...

2026-01-17 07:52:27 I [flux_generator:173] - [FluxGenerator] Background generation completed
2026-01-17 07:52:27 I [flux_generator:322] - [FluxGenerator] Injecting features via Inpainting...
2026-01-17 07:52:27 I [flux_generator:322] - [FluxGenerator] Injecting features via Inpainting...
2026-01-17 07:52:27 I [flux_generator:102] - [FluxGenerator] Loading FLUX Inpaint pipeline...
2026-01-17 07:52:27 I [flux_generator:102] - [FluxGenerator] Loading FLUX Inpaint pipeline...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The module 'FluxTransformer2DModel' has been loaded in `bitsandbytes` 8bit and moving it to cpu via `.to()` is not supported. Module is still on cuda:0.


2026-01-17 07:52:31 I [flux_generator:115] - [FluxGenerator] Inpaint pipeline ready
2026-01-17 07:52:31 I [flux_generator:353] - [FluxGenerator] Injecting features: prompt='
Crystal clear, ultra-sharp dried yellow corvina f...', strength=0.3
2026-01-17 07:52:31 I [flux_generator:357] -  [FluxGenerator] prompt='
Crystal clear, ultra-sharp dried yellow corvina fish (nogari) on a white plate. 
Photorealistic detail with visible texture, char marks, and natural sheen. 
Sharp focus on product, contrasting with blurred cartoon background. 
Professional food photography lighting, 8k, masterpiece quality.
' 
2026-01-17 07:52:31 I [flux_generator:358] -  [FluxGenerator] negative_prompt='blurry product, soft focus product, cartoon fish, unrealistic' 
2026-01-17 07:52:31 I [flux_generator:353] - [FluxGenerator] Injecting features: prompt='
Crystal clear, ultra-sharp dried yellow corvina f...', strength=0.3
2026-01-17 07:52:31 I [flux_generator:357] -  [FluxGenerator] prompt='
Crystal clear, ultra